# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [3]:
weather_data = pd.read_csv('../output_data/city_data.csv')
weather_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date
0,bonthe,7.53,-12.51,81.27,78,70,7.09,SL,1603475575
1,lata,40.16,-8.33,63.00,68,41,4.00,PT,1603475575
2,beyneu,45.32,55.20,40.82,65,3,9.95,KZ,1603475575
3,roros,62.57,11.38,17.60,92,0,1.12,NO,1603475575
4,buritizeiro,-17.35,-44.96,74.73,80,89,4.70,BR,1603475576
...,...,...,...,...,...,...,...,...,...
543,provideniya,64.38,-173.30,33.30,92,100,13.44,RU,1603475704
544,rodrigues alves,-7.74,-72.65,97.83,35,88,3.42,BR,1603475704
545,ihosy,-22.40,46.12,66.47,78,12,11.48,MG,1603475705
546,marawi,8.00,124.29,73.99,93,91,5.01,PH,1603475705


In [4]:
type(weather_data['Cloudiness'][1])

numpy.int64

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [5]:
#configure gmaps
gmaps.configure(api_key=g_key)

In [6]:
#Create Heatmap of Humidity
locations = weather_data[['Lat', 'Lng']]
weights = weather_data['Humidity']
fig = gmaps.figure()
fig.add_layer(gmaps.heatmap_layer(locations, weights=weights))
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
    * My criteria: between 72 and 76 degrees, non cloudy, windspeed less than 10 mph, humidity less than 70%
* Drop any rows will null values.

In [21]:
ideal_cities = weather_data[(weather_data['Max Temp'] >=72) & (weather_data['Max Temp'] <=76 ) & (weather_data['Cloudiness'] <= 0)
                            & (weather_data['Windspeed']<10) & (weather_data['Humidity']<70)]

#take the latitude and longitude and concatenate them into coordinates for google places API search 
ideal_cities['coordinates'] = ideal_cities['Lat'].astype(str).str.cat(ideal_cities['Lng'].astype(str), sep =", ")
ideal_cities

<ipython-input-21-3001b548ddb2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ideal_cities['coordinates'] = ideal_cities['Lat'].astype(str).str.cat(ideal_cities['Lng'].astype(str), sep =", ")


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Windspeed,Country,Date,coordinates
201,khed brahma,24.03,73.05,74.88,30,0,1.72,IN,1603475623,"24.03, 73.05"
293,moindou,-21.69,165.68,75.20,64,0,5.82,NC,1603475646,"-21.69, 165.68"
296,general roca,-39.03,-67.58,75.20,53,0,5.82,AR,1603475647,"-39.03, -67.58"
297,abu samrah,35.30,37.18,72.05,22,0,6.96,SY,1603475647,"35.3, 37.18"
387,sur,22.57,59.53,73.96,61,0,4.54,OM,1603475667,"22.57, 59.53"


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [35]:
hotel_df = pd.DataFrame()
hotel_df['Hotel Name'] = ""
hotel_df['City'] = ""
hotel_df['Country'] = ""
hotel_df['lat'] = ""
hotel_df['lng'] = ""

In [14]:
# test hit the google places API to get the first result and get the information into the database 
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "-21.69, 165.68",  # Khed Brahma
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
    #"keyword": "hotel",
}
    
response = requests.get(base_url, params=params).json() 


In [15]:
response['results'][0]['name']

'Gite de la RYAWA'

In [39]:
response['results'][0]['geometry']['location']['lat']

22.5664354

In [18]:
response[''] 

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': -21.6713994, 'lng': 165.6791161},
    'viewport': {'northeast': {'lat': -21.6701399697085,
      'lng': 165.6805614802915},
     'southwest': {'lat': -21.6728379302915, 'lng': 165.6778635197085}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Gite de la RYAWA',
   'photos': [{'height': 1134,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/106270599816920865265">boatpasstation NC</a>'],
     'photo_reference': 'ATtYBwInjcQ_piIKW1RdN7hxXX9Vmv565omUH4gVXKLS7jTj4vguBo358cVIxAO-Omfq_m8_1G5pO_n6cMjWE0SCtanQMc3_OgYxMDzxDLZtenK5TmUYEFP1PXQBaIhe_HF_TATu27f7xXwqcZcUixc9RI1c4ztuxhn8oj_rUh7hF7BYTC-5',
     'width': 850}],
   'place_id': 'ChIJzcjsq1KfnW4Rl2CwhQRyxSE',
   'plus_code': {'compound_code': '8MHH+CJ Mouidou, New Caledonia',
    'global_code': '5VC78MHH+CJ'},
   'rating': 5,
   'reference': 'ChIJzcjs

In [40]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {
    "location": "",  # philadelphia coords,
    "radius": "5000",
    "type": "lodging",
    "key": g_key,
#     "keyword": "Hotel",
}
# use iterrows to iterate through pandas dataframe
for index, row in ideal_cities.iterrows():

    # get coordinates from df
    coordinates = row['coordinates']

    # add coordinates to params dict
    params["location"] = coordinates

    # assemble url and make API request
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    try:
        print(f"Closest hotel to {row['City']} is {results[0]['name']}.")
        
        hotel_df.loc[index, 'Hotel Name'] = results[0]['name']
        hotel_df.loc[index, 'City'] = row['City']
        hotel_df.loc[index, 'Country'] = row['Country']
        hotel_df.loc[index, 'lat'] = results[0]['geometry']['location']['lat']
        hotel_df.loc[index, 'lng'] = results[0]['geometry']['location']['lng']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")

Closest hotel to khed brahma is UMIYA NASTA HOUSE.
------------
Closest hotel to moindou is Gite de la RYAWA.
------------
Closest hotel to general roca is Hotel El Recreo.
------------
Missing field/result... skipping.
------------
Closest hotel to sur is Sur Plaza Hotel.
------------


In [41]:
#display the updated data frame to ensure we got the information
hotel_df

,Hotel Name,City,Country,lat,lng
201,UMIYA NASTA HOUSE,khed brahma,IN,24.0311,73.0483
293,Gite de la RYAWA,moindou,NC,-21.6714,165.679
296,Hotel El Recreo,general roca,AR,-39.04,-67.6124
387,Sur Plaza Hotel,sur,OM,22.5664,59.4959


In [51]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
""" 
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(Hotel=row['Hotel Name'], City=row['City'], Country=row['Country']) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "lng"]]

In [52]:
hotel_df['locations'] = hotel_df.apply(lambda x: (x['lat'], x['lng']), axis = 1)
hotel_df

,Hotel Name,City,Country,lat,lng,locations
201,UMIYA NASTA HOUSE,khed brahma,IN,24.0311,73.0483,"(24.0310886, 73.0483465)"
293,Gite de la RYAWA,moindou,NC,-21.6714,165.679,"(-21.6713994, 165.6791161)"
296,Hotel El Recreo,general roca,AR,-39.04,-67.6124,"(-39.04000319999999, -67.6124162)"
387,Sur Plaza Hotel,sur,OM,22.5664,59.4959,"(22.5664354, 59.4958585)"


In [54]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(marker_locations, info_box_content=hotel_info)
fig.add_layer(markers)
# Display figure
fig

Figure(layout=FigureLayout(height='420px'))